In [2]:
import numpy as np
import codecs
import pandas as pd

In [3]:
data = pd.read_csv('train.csv')

In [4]:
mapping = {}

In [5]:
tid1s = data['tid1'].tolist()
tid2s = data['tid2'].tolist()
title1_zhs = data['title1_zh'].tolist()
title2_zhs = data['title2_zh'].tolist()
title1_ens = data['title1_en'].tolist()
title2_ens = data['title2_en'].tolist()
labels = data['label'].tolist()

In [7]:
for tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label in zip(tid1s,tid2s,title1_zhs,title2_zhs,title1_ens,title2_ens,labels):
    mapping[(tid1,tid2)] = label

In [8]:
test = pd.read_csv('test.csv')

In [13]:
ids = test['id'].tolist()
tid1s = test['tid1'].tolist()
tid2s = test['tid2'].tolist()
with open('pred.csv') as f:
    line = f.readlines()
count = 0
with open('pred_pos.csv','w') as f:    
    f.write(line[0])
    for i,(gid,tid1,tid2) in enumerate(zip(ids,tid1s,tid2s)):
        temp = None
        if((tid1,tid2) in mapping):
            temp = label
        if((tid2,tid1) in mapping):
            temp = label
        
        if(temp is not None):
            f.write("{0},{1}\n".format(gid,temp))
            count +=1
        else:
            f.write("{0},{1}".format(gid,line[i+1].split(',')[1]))
print(count)

138


In [15]:
exist = set()
mapping = {}
strange = set()

In [16]:
tid1s = data['tid1'].tolist()
tid2s = data['tid2'].tolist()
title1_zhs = data['title1_zh'].tolist()
title2_zhs = data['title2_zh'].tolist()
title1_ens = data['title1_en'].tolist()
title2_ens = data['title2_en'].tolist()
labels = data['label'].tolist()

In [17]:
class graph:
    def __init__(self):
        self.data = {}
        self.dis_path = {}
        self.un = {}
        
    def add_link(self,a,b):
        def add(a,b):
            try:
                self.data[a].append(b)
            except:
                self.data[a] = [b]
        add(a,b)
        add(b,a)
    
    def add_dis(self,a,b):
        def dis(a,b):
            if(b not in self.path):
                self.path[b] = [b]        
            if(a in self.path[b]):
                strange.add((a,b))
                return True
            try:
                self.dis_path[a].extend(self.path[b])
            except KeyError as e:
                self.dis_path[a] = self.path[b]
            return False
        
        if(dis(a,b)):
            return 
        dis(b,a)
            
    def dfs_all(self):
        def dfs(a,check):
            ans = [a]
            check.add(a)
            for p in self.data[a]:
                if(p not in check):
                    ans.extend(dfs(p,check))
            return ans
            
            
        self.path = {}
        
        for root in self.data:
            temp = dfs(root,set())
            
            self.path[root] = temp
            
    def add_un(self,a,b):
        def un(a,b):
            
            if((b in self.path and a in self.path[b]) or (b in self.dis_path and a in self.dis_path[b])):
                strange.add((a,b))
                return True
            
            try:
                self.un[a].append(b)
            except:
                self.un[a] = [b]
            return False
        
        if(un(a,b)):
            return 
        un(b,a)
        
            

In [18]:
g = graph()
#first build up the disjoint set
count = 0
for tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label in zip(tid1s,tid2s,title1_zhs,title2_zhs,title1_ens,title2_ens,labels):
    if(tid1 not in exist):
        exist.add(tid1)
        mapping[tid1] = [title1_zhs,title1_ens]
    if(tid2 not in exist):
        exist.add(tid2)
        mapping[tid2] = [title2_zhs,title2_ens]
    
    if(tid2==42543 or tid1==42543):
        continue
    if(tid1==108313 and tid1==108312):
        continue
        
    if(label=='agreed'):
        count += 1
        g.add_link(tid1,tid2)
g.dfs_all()

In [19]:
after_count = 0
for key in g.path:
    after_count += len(g.path[key])-1
    
len(g.path),count,after_count//2

(59018, 92966, 413301)

In [20]:
count = 0
for tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label in zip(tid1s,tid2s,title1_zhs,title2_zhs,title1_ens,title2_ens,labels):
    if(label=='disagreed'):
        if(tid1 == tid2):
            continue
        count += 1
        g.add_dis(tid1,tid2)


In [21]:
after_count = 0
for key in g.dis_path:
    after_count += len(g.dis_path[key])
    
len(g.dis_path),count,after_count//2

(66381, 8256, 166692)

In [22]:
count = {'unrelated':0,'agreed':0,'disagreed':0}
for tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label in zip(tid1s,tid2s,title1_zhs,title2_zhs,title1_ens,title2_ens,labels):
    if(tid1 == tid2):
        count[label] += 1
print(count)

{'unrelated': 96, 'agreed': 4, 'disagreed': 10}


In [23]:
count = 0
for tid1,tid2,title1_zh,title2_zh,title1_en,title2_en,label in zip(tid1s,tid2s,title1_zhs,title2_zhs,title1_ens,title2_ens,labels):
    if(label=='unrelated'):
        if(tid1 == tid2):
            continue
        count += 1
        g.add_un(tid1,tid2)

In [24]:
after_count = 0
for key in g.un:
    after_count += len(g.un[key])-1
    
len(g.un),count,after_count//2

(66381, 219217, 135845)

In [30]:
ids = test['id'].tolist()
tid1s = test['tid1'].tolist()
tid2s = test['tid2'].tolist()
with open('pred.csv') as f:
    line = f.readlines()
count = [0,0,0,0,0]
with open('pred_pos_pos.csv','w') as f:    
    f.write(line[0])
    for i,(gid,tid1,tid2) in enumerate(zip(ids,tid1s,tid2s)):
        temp = None
        
        if((tid1 in g.path and tid2 in g.path[tid1]) or (tid1 in g.path and tid2 in g.path[tid1])):
            #temp = 'agreed'
            count[0] += 1
        elif((tid1 in g.dis_path and tid2 in g.dis_path[tid1]) or (tid1 in g.dis_path and tid2 in g.dis_path[tid1])):
            temp = 'disagreed'
            count[1] += 1
        elif((tid1 in g.un and tid2 in g.un[tid1]) or (tid1 in g.un and tid2 in g.un[tid1])):
            temp = 'unrelated'
            count[2] += 1
        if(tid1 == tid2):
            temp = 'unrelated'
            count[3] += 1
        if(tid2==42543):
            temp = 'agreed'
            count[4] += 1
        
        if(temp is not None):
            f.write("{0},{1}\n".format(gid,temp))
        else:
            f.write("{0},{1}".format(gid,line[i+1].split(',')[1]))
print(count)

[6867, 5, 44, 44, 1]
